<a href="https://colab.research.google.com/github/Shyam-Khokhariya/ML-Python/blob/master/C15_CNN_Mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

In [0]:
from google.colab import files
uploaded=files.upload()

In [0]:
mnist=np.load("mnist_scaled.npz")
X_train,y_train,X_test,y_test=[mnist[f] for f in mnist.files]

In [4]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [5]:
X_valid,y_valid=X_train[50000:,:],y_train[50000:]
X_train,y_train=X_train[:50000,:],y_train[:50000]
print(X_train.shape,y_train.shape)
print(X_valid.shape,y_valid.shape)
print(X_test.shape,y_test.shape)

(50000, 784) (50000,)
(10000, 784) (10000,)
(10000, 784) (10000,)


In [0]:
def batch_generator(X,y,batch_size=64,shuffle=False,random_seed=None):
  idx=np.arange(y.shape[0])
  if shuffle:
    rng=np.random.RandomState(random_seed)
    rng.shuffle(idx)
    X=X[idx]
    y=y[idx]
  for i in range(0,X.shape[0],batch_size):
    yield (X[i:i+batch_size,:],y[i:i+batch_size])

In [0]:
mean_val=np.mean(X_train,axis=0)
std_val=np.std(X_train)

X_train_centered=(X_train-mean_val)/std_val
X_valid_centered=(X_valid-mean_val)/std_val
X_test_centered=(X_test-mean_val)/std_val

In [0]:
def conv_layer(input_tensor,name,
               kernel_size,n_output_channels,
               padding_mode="SAME",strides=(1,1,1,1)):
  with tf.variable_scope(name):
    input_shape=input_tensor.get_shape().as_list()
    n_input_channels=input_shape[-1]
    weight_shape=list(kernel_size)+[n_input_channels,n_output_channels]
    print(weight_shape)
    weights=tf.get_variable(name="_weights",shape=weight_shape)
    print("Weights : ",weights)
    biases=tf.get_variable(name="_biases",initializer=tf.zeros(shape=[n_output_channels]))
    print("Biases : ",biases)
    
    conv=tf.nn.conv2d(input=input_tensor,filter=weights,strides=strides,padding=padding_mode)
    
    print("Conv : ",conv)
    
    conv=tf.nn.bias_add(conv,biases,name="net_pre-activation")
    
    print("Conv Bias Addition :",conv)
    
    conv=tf.nn.relu(conv,name="Activation")
    
    print("ACtivated Conv : ",conv)
    
    return conv

In [16]:
g=tf.Graph()
with g.as_default():
  x=tf.placeholder(tf.float32,shape=[None,28,28,1])
  conv_layer(x,name="conv_test",kernel_size=(3,3),n_output_channels=32)

del g,x

[3, 3, 1, 32]
Weights :  <tf.Variable 'conv_test/_weights:0' shape=(3, 3, 1, 32) dtype=float32_ref>
Biases :  <tf.Variable 'conv_test/_biases:0' shape=(32,) dtype=float32_ref>
Conv :  Tensor("conv_test/Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Conv Bias Addition : Tensor("conv_test/net_pre-activation:0", shape=(?, 28, 28, 32), dtype=float32)
ACtivated Conv :  Tensor("conv_test/Activation:0", shape=(?, 28, 28, 32), dtype=float32)


In [0]:
def fc_layer(input_tensor,name,n_output_units,activation_fn=None):
  with tf.variable_scope(name):
    input_shape=input_tensor.get_shape().as_list()[1:]
    n_input_units=np.prod(input_shape)
    if len(input_shape)>1:
      input_tensor=tf.reshape(input_tensor,shape=(-1,n_input_units))
    weights_shape=[n_input_units,n_output_units]
    weights=tf.get_variable(name="_weights",shape=weights_shape)
    print("Weights : ",weights)
    
    biases=tf.get_variable(name="_biases",initializer=tf.zeros(shape=[n_output_units]))
    print("Biases : ",biases)
    
    layer=tf.matmul(input_tensor,weights)
    print("Layer : ",layer)
    
    layer=tf.nn.bias_add(layer,biases,name="net_pre-activation")
    print("Biased added layer : ",layer)
    
    if activation_fn is None:
      return layer
    
    layer=activation_fn(layer,name="activation")
    print("1 layer output : ",layer)
    return layer

In [19]:
g1=tf.Graph()
with g1.as_default():
  x=tf.placeholder(tf.float32,shape=[None,28,28,1])
  fc_layer(x,name="fctest",n_output_units=32,activation_fn=tf.nn.relu)
del g1,x

Weights :  <tf.Variable 'fctest/_weights:0' shape=(784, 32) dtype=float32_ref>
Biases :  <tf.Variable 'fctest/_biases:0' shape=(32,) dtype=float32_ref>
Layer :  Tensor("fctest/MatMul:0", shape=(?, 32), dtype=float32)
Biased added layer :  Tensor("fctest/net_pre-activation:0", shape=(?, 32), dtype=float32)
1 layer output :  Tensor("fctest/activation:0", shape=(?, 32), dtype=float32)


In [0]:
def build_cnn():
  tf_x=tf.placeholder(tf.float32,shape=[None,784],name="tf_x")
  tf_y=tf.placeholder(tf.int32,shape=[None],name="tf_y")
  
  tf_x_image=tf.reshape(tf_x,shape=[-1,28,28,1],name="tf_x_reshaped")
  tf_y_onehot=tf.one_hot(indices=tf_y,depth=10,dtype=tf.float32,name="tf_y_onehot")
  
  #1st Layer:Conv1
  print("-----Building 1st Layer-----\n")
  h1=conv_layer(tf_x_image,name="conv_1",kernel_size=(5,5),padding_mode="VALID",n_output_channels=32)
   
  #MAX POOLING
  h1_pool=tf.nn.max_pool(h1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
  
  #2nd Layer:Conv2
  print("\n-----Building 2nd Layer-----\n")
  h2=conv_layer(h1_pool,name="conv_2",kernel_size=(5,5),padding_mode="VALID",n_output_channels=64)
  
  #MAX POOLING
  h2_pool=tf.nn.max_pool(h2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
  
  #3rd Layer:Fully Connected
  print("\n-----Building 3rd Layer-----\n")
  h3=fc_layer(h2_pool,name="fc_3",n_output_units=1024,activation_fn=tf.nn.relu)
  
  #DROPOUT
  keep_prob=tf.placeholder(tf.float32,name="fc_keep_prob")
  h3_drop=tf.nn.dropout(h3,keep_prob=keep_prob,name="dropout_layer")
  
  ##4th Layer:Fully Connected (Linear Activation)
  print("\n-----Building 4th Layer-----\n")
  h4=fc_layer(h3_drop,name="fc_4",n_output_units=10,activation_fn="None")
  
  ##Prediction
  prediction={
      'probabilities':tf.nn.softmax(h4,name="probabilities"),
      'labels':tf.cast(tf.argmax(h4,axis=1),tf.int32,name="labels")
  }
  
  ##Visualize graph with tensorboard
  
  ##LossFunction and Optimization
  cross_entropy_loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=h4,labels=tf_y_onehot),name="cross_entropy_loss")
  
  ##Optimizer
  optimizer=tf.train.AdamOptimizer(learning_rate)
  optimizer=optimizer.minimize(cross_entropy_loss,name="train_op")
  
  #Computing prediction Accuracy
  correct_prediction=tf.equal(prediction['labels'],tf_y,name="correct_preds")
  accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32),name="accuracy")
  

In [0]:
def save(saver,sess,epoch,path="./model/"):
  if not os.path.isdir(path):
    os.makedirs(path)
  print("Saver Model in %s" %path)
  saver.save(sess,os.path.join(path,"cnn-model.ckpt"),global_step=epoch)


In [0]:
def load(saver,sess,path,epoch):
  print("Loading model from %s" %path)
  saver.restore(sess,os.path.join(path,"cnn-model.ckpt-%d"%epoch))

In [0]:
def train(sess,training_set,validation_set=None,
         initialization=True,epochs=20,shuffle=True,
         dropout=0.5,random_seed=None):
  
  X_data=np.array(training_set[0])
  y_data=np.array(training_set[1])
  training_loss=[]
  
  #initializing variables
  if initialize:
    sess.run(tf.global_variables_initializer)
  
  np.random.seed(random_seed)
  
  for epoch in range(1,epoch+1):
    batch_gen=batch_generator(X_data,y_data,shuffle=shuffle)
    avg_loss=0.0
    for i,(batch_x,batch_y) in enumerate(batch_gen):
      feed={"tf_x:0":batch_x,
           "tf_y:0":batch_y,
           "fc_keep_prob:0":dropout}
      loss,_=sess.run(["cross_entropy_loss:0","train_op"],feed_dict=feed)
      avg_loss+=loss
    training_loss.append(avg_loss/(i+1))
    print(" Epoch %02d Training Average Loss : %7.3f"%(epoch,avg_loss),end=" ")
    if validation_set is not None:
      feed={"tf_x:0":validation_set[0],
           "tf_y:0":validation_set[1],
           "fc_keep_prob:0":1.0}
      valid_acc=sess.run("Accuracy : 0",feed_dict=feed)
      print(" Validation Accuracy : %7.3f" %valid_acc)
    else:
      print()
      

In [0]:
def predict(sess,X_test,return_prob=False):
  feed={"tf_x:0":X_test,
       "fc_keep_prob:0":1.0}
  if return_prob:
    return sess.run("probabilities:0",feed_dict=feed)
  else:
    return sess.run("labels:0",feed_dict=feed)